<H2> Intro to Applied NLP -- Lab 8 + HW 6 </H2>

<H3> Similarity Measures, Revisited </H3>

<H4> Name: YUEYANG LIU</H4>

In [0]:
# the 20 newsgroups dataset is common these days -- let's start here
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train') #takes a bit...
newsgroups_test = fetch_20newsgroups(subset='test')
from collections import Counter

In [0]:
# let's see how the data is categorized
newsgroups_test.target_names==newsgroups_train.target_names

True

In [0]:
# let's look at one entry of the data
print(newsgroups_train.data[9])

From: kerr@ux1.cso.uiuc.edu (Stan Kerr)
Subject: Re: Sigma Designs Double up??
Article-I.D.: ux1.C52u8x.B62
Organization: University of Illinois at Urbana
Lines: 29

jap10@po.CWRU.Edu (Joseph A. Pellettiere) writes:


>	I am looking for any information about the Sigma Designs
>	double up board.  All I can figure out is that it is a
>	hardware compression board that works with AutoDoubler, but
>	I am not sure about this.  Also how much would one cost?

I've had the board for over a year, and it does work with Diskdoubler,
but not with Autodoubler, due to a licensing problem with Stac Technologies,
the owners of the board's compression technology. (I'm writing this
from memory; I've lost the reference. Please correct me if I'm wrong.)

Using the board, I've had problems with file icons being lost, but it's
hard to say whether it's the board's fault or something else; however,
if I decompress the troubled file and recompress it without the board,
the icon usually reappears. Because of the

In [0]:
# let's look at a few entries of the data categories
newsgroups_train.target[0:5]

array([ 7,  4,  4,  1, 14])

In [0]:
# let's examine how many documents there are!
len(newsgroups_train.data)

11314


<H3> The usual pre-processing </H3>

1. Create a function that cleans the text data. Be sure to remove things like email addresses/internet domains.




In [0]:
import re
def clean_text(text):
    # url email
    text=re.sub(r'(((https|http|ftp|rtsp|mms)?:\/\/)[^\s]+)|(\w[-\w.+]*@([A-Za-z0-9][-A-Za-z0-9]+\.)+[A-Za-z]{2,14})', "", text)

    # acronym
    #text = re.sub(r"\n", "", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"can\’t", "can not", text)
    text = re.sub(r"cannot", "can not ", text)
    text = re.sub(r"what\'s", "what is", text)
    text = re.sub(r"What\'s", "what is", text)
    text = re.sub(r"\'ve ", " have ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"I\'m", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r",000", '000', text)
    text = re.sub(r"\'s|s\'|’s|\'|#|\*|:|>", "", text)
    
    # symbol replacement
    text = re.sub(r"&", " and ", text)
    text = re.sub(r"\|", " or ", text)
    text = re.sub(r"\$", " dollar ", text)
    text = re.sub(r"(\.|,|\?|;|!|\–|\-|\(|\))", " ", text)
    #text = re.sub(r"(\”)", " ", text)
    text=text.lower()
    # remove extra space
    text = ' '.join(text.split())

    return text

In [0]:
data,tag = fetch_20newsgroups(shuffle=True, 
                              #random_state=1,
                             remove=('headers', 'footers', 'quotes'),
                             return_X_y=True,
                              subset='train')
clean_data=[]
for i in data[:1000]: clean_data.append(clean_text(i))
data_samples = clean_data
data_samples=data[:1000]
#sampletag=tag[:1000]
sampletag=tag[:1000]

In [0]:
tag[0:5]

array([ 7,  4,  4,  1, 14])

In [0]:
sampletag[0:5]

array([ 7,  4,  4,  1, 14])

<H3> Topic Modeling </H3>

2. Use sklearn's CountVectorizer and TFIDFVectorizer to transform the text data to a matrix:

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
n_samples = len(data)
n_features = 1000
n_components = 10
n_top_words = 20

In [0]:
tfidf_vectorizer = TfidfVectorizer(
                                  max_df=0.85, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(data_samples)


In [0]:
tf_vectorizer = CountVectorizer(
                                max_df=0.85, min_df=2,
                                #max_features=n_features,
                                stop_words='english')

tf = tf_vectorizer.fit_transform(data_samples)

3. Follow along this code, to apply NMF to the corpus:

https://scikit-learn.org/stable/auto_examples/applications/plot_topics_extraction_with_nmf_lda.html#sphx-glr-auto-examples-applications-plot-topics-extraction-with-nmf-lda-py

In [0]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [0]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

In [0]:
nmf = NMF(n_components=n_components, #must add commpoenents num or high ram use
          random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
nmf_ma=nmf.fit_transform(tfidf)


Topics in NMF model (Frobenius norm):
Topic #0: don know like people good time ve really use way want does things say right problem new ll used make
Topic #1: thanks mail info information advance address appreciated anybody looking greatly hi help need appreciate subject knows know edu does mac
Topic #2: god jesus truth faith christian says believe john sin hear nature father man people christ christianity did follow christians bible
Topic #3: armenian armenians turkish genocide soviet government armenia people turks russian muslim population million women argic serdar children war kurds 000
Topic #4: windows files program file ftp use run pc image dos software microsoft code called disk nt write using data win
Topic #5: just tell oh version work did guess new water used reading speed bit wanted simms little try turkish kind price
Topic #6: sale offer 00 condition asking cable disks best price interested 15 make new email excellent edu included cd modem respond
Topic #7: game team run

4. Compare NMF Results with LSA Results, when using TFIDF Vectorizer, with n-ranges: (1,1), (1,2), (2,3). Which yield the best (empirical) results? You may have to play around with the TFIDF Vectorizer's min/max df parameter. (Don't forget about the stopword removal!)


In [0]:
from sklearn.decomposition import TruncatedSVD

terms = tfidf_vectorizer.get_feature_names()
lsa = TruncatedSVD(n_components)               
X2 = lsa.fit_transform(tfidf)
n_pick_docs= 1
topic_docs_id = [X2[:,t].argsort()[:-(n_pick_docs+1):-1] for t in range(n_components)]
topic_keywords_id = [lsa.components_[t].argsort()[:-(n_top_words+1):-1] for t in range(n_components)]
print("Topics in LSA")
for t in range(n_components):
  print("topic #%d:" % t," ".join(terms[topic_keywords_id[t][j]] for j in range(n_top_words)))

Topics in LSA
topic #0: don just know think like people good time use ve does thanks really new way did god want problem say
topic #1: thanks windows mail files program use file card information edu info software advance pc looking address com problem need anybody
topic #2: god thanks jesus mail truth com people christian says does know windows faith believe information address program advance greatly christianity
topic #3: armenian armenians turkish government people com genocide thanks soviet edu information mail address armenia new 000 turks 00 children russian
topic #4: thanks just mail good info looking sale game address god offer season information team oh tell think bike price year
topic #5: just problem card turkish armenian armenians memory thanks god video simms used did jesus mac soviet genocide info ram new
topic #6: jesus god sale new 00 game make offer year good 10 games condition power 20 15 12 team john asking
topic #7: think com just edu year data new read problem prog

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                                  max_df=0.95, min_df=2,
                                   #max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)
terms = tfidf_vectorizer.get_feature_names()
lsa = TruncatedSVD(n_components)               
X2 = lsa.fit_transform(tfidf)
n_pick_docs= 1
topic_docs_id = [X2[:,t].argsort()[:-(n_pick_docs+1):-1] for t in range(n_components)]
topic_keywords_id = [lsa.components_[t].argsort()[:-(n_top_words+1):-1] for t in range(n_components)]
print("Topics in LSA")
for t in range(n_components):
  print("topic #%d:" % t," ".join(terms[topic_keywords_id[t][j]] for j in range(n_top_words)))

Topics in LSA
topic #0: don know just people think like good use time does ve god thanks did really way new problem want jesus
topic #1: god jesus people truth think believe armenian christian armenians say don said faith argument turkish bible did man religion true
topic #2: cadre dsl intellect geb shameful surrender shameful edu shameful geb cadre cadre dsl pitt dsl surrender soon geb n3jxp skepticism chastity intellect chastity banks n3jxp skepticism chastity n3jxp gordon banks pitt pitt edu
topic #3: armenian armenians turkish genocide soviet government armenia turks 000 russian people soviet armenian armenian government population muslim war killed million turkey women
topic #4: god jesus armenian windows armenians turkish thanks program mail file files genocide christian people soviet truth info faith image address
topic #5: thanks mail 00 sale offer god jesus address mail thanks looking game good best offer info year com information condition price interested
topic #6: jesus car

In [0]:
nmf = NMF(n_components=n_components, #must add commpoenents num or high ram use
          random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
nmf_ma=nmf.fit_transform(tfidf)


Topics in NMF model (Frobenius norm):
Topic #0: don know think just people like good ve god time really does way want say did things use believe right
Topic #1: simms use pin machine memory 80ns 80ns simms crashes bk 50mhz bank apple 30 mac 70ns lc meg slower faster need
Topic #2: dsl pitt cadre dsl dsl shameful surrender geb cadre edu shameful intellect geb surrender soon cadre shameful geb skepticism chastity banks n3jxp n3jxp skepticism n3jxp chastity intellect chastity gordon banks pitt pitt edu
Topic #3: armenian armenians turkish genocide soviet armenia turks people russian soviet armenian government armenian government muslim turkey soviet armenia population million 000 kurds women
Topic #4: jesus god john matthew sin faith king father kingdom apostles jesus did says death punishment people messiah did david passage man
Topic #5: thanks mail mail thanks address howard looking thanks advance com info advance reader font needed thanks knows let know line information know mail add

In [0]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3),
                                  max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(data_samples)
terms = tfidf_vectorizer.get_feature_names()
lsa = TruncatedSVD(n_components)               
X2 = lsa.fit_transform(tfidf)
n_pick_docs= 1
topic_docs_id = [X2[:,t].argsort()[:-(n_pick_docs+1):-1] for t in range(n_components)]
topic_keywords_id = [lsa.components_[t].argsort()[:-(n_top_words+1):-1] for t in range(n_components)]
print("Topics in LSA")
for t in range(n_components):
  print("topic #%d:" % t," ".join(terms[topic_keywords_id[t][j]] for j in range(n_top_words)))

Topics in LSA
topic #0: don just know think like people good time use ve does thanks really new way did god want problem say
topic #1: thanks windows mail files program use file card edu information info advance software pc looking address com problem anybody using
topic #2: god jesus thanks mail truth com christian windows says does people faith believe information know advance address greatly question christianity
topic #3: armenian armenians turkish government thanks people com genocide soviet edu information mail address 00 000 armenia new russian turks children
topic #4: thanks just mail good looking game god season sale info team oh address tell offer hockey information great 00 year
topic #5: just problem simms card god new turkish did memory work used tell jesus armenian thanks armenians mac info video water
topic #6: don know thanks just com don know mail window want tell right think turkish armenian people government way armenians let info
topic #7: think com problem don new 

In [0]:
nmf = NMF(n_components=n_components, #must add commpoenents num or high ram use
          random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)
nmf_ma=nmf.fit_transform(tfidf)


Topics in NMF model (Frobenius norm):
Topic #0: don know like people good time ve use really want way does things say right problem new ll did make
Topic #1: thanks mail info information advance address appreciated anybody looking greatly hi help need appreciate subject edu knows know does mac
Topic #2: god jesus truth faith christian says believe sin john hear nature man people father christ christianity did christians follow bible
Topic #3: armenian armenians turkish genocide soviet government armenia people turks soviet armenian russian armenian government muslim population million argic women serdar serdar argic 000
Topic #4: windows files program file ftp use run pc image dos microsoft software code called disk write nt using data win
Topic #5: just tell oh version work did guess new water used reading speed bit wanted simms little try turkish price kind
Topic #6: sale offer 00 condition asking cable disks best price interested new 15 make email edu excellent included cd modem re

according to the result the (1:2)ngream waorks well if we raise to (1:3) we can't see a improvement and during the raise of ngram the NMF don't see a improvement.

<H3> Information Retrieval </H3>  
5. Use a clustering algorithm to group documents:  

a) by the Count Vectorized Form  
b) by the TF-IDF Vectorized output  
c) by the similarity of their NMF topic distribution, for the ngram ranges identified in step 6.   

Compare results from Agglomerative Clustering and k-means clustering. Also experiment with the number of clusters you expect the algorithm to find. 

Hint 1: the number of clusters should be similar to the amount of categories in this dataset.

Hint 2: There are at least 6 experiments you have to compare here. Be sure to keep track of them all appropriately.

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html



In [0]:
from sklearn import metrics  
def calculate_result(actual,pred):  
  m_precision = metrics.precision_score(actual,pred, average='weighted')
  m_recall = metrics.recall_score(actual,pred,average='weighted')
  print ('predict info:'  )
  print ('precision:{0:.3f}'.format(m_precision)  )
  print ('recall:{0:0.3f}'.format(m_recall))
  print ('f1-score:{0:.3f}'.format(metrics.f1_score(actual,pred,average='weighted')))

In [0]:
def km_j(labels,km_labels_,tfidf):
  print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels, km_labels_))
  print("Completeness: %0.3f" % metrics.completeness_score(labels, km_labels_))
  print("V-measure: %0.3f" % metrics.v_measure_score(labels, km_labels_))
  print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(tfidf, km_labels_, sample_size=1000))
  

In [0]:
from sklearn.cluster import AgglomerativeClustering
cv_ac = AgglomerativeClustering(n_clusters=20).fit(tf.toarray())
cv_ac_p=cv_ac.fit_predict(tf.toarray())
tf_ac = AgglomerativeClustering(n_clusters=20).fit(tfidf.toarray())
tf_ac_p=tf_ac.fit_predict(tfidf.toarray())
nmf_ac = AgglomerativeClustering(n_clusters=20).fit(nmf_ma)
nmf_ac_p=nmf_ac.fit_predict(nmf_ma)
lsa_ac= AgglomerativeClustering(n_clusters=20).fit(X2)
lsa_ac_p=lsa_ac.fit_predict(X2)

In [0]:
km_j(sampletag,cv_ac_p,tf)
calculate_result(sampletag,cv_ac_p)

Homogeneity: 0.039
Completeness: 0.346
V-measure: 0.071
Silhouette Coefficient: 0.554
predict info:
precision:0.003
recall:0.045
f1-score:0.005


In [0]:
km_j(sampletag,tf_ac_p,tfidf)
calculate_result(sampletag,tf_ac_p)

Homogeneity: 0.208
Completeness: 0.302
V-measure: 0.246
Silhouette Coefficient: 0.015
predict info:
precision:0.057
recall:0.046
f1-score:0.034


In [0]:
km_j(sampletag,nmf_ac_p,nmf_ma)
calculate_result(sampletag,nmf_ac_p)

Homogeneity: 0.188
Completeness: 0.242
V-measure: 0.212
Silhouette Coefficient: 0.269
predict info:
precision:0.046
recall:0.063
f1-score:0.046


In [0]:
km_j(sampletag,lsa_ac_p,X2)
calculate_result(sampletag,lsa_ac_p)

Homogeneity: 0.258
Completeness: 0.276
V-measure: 0.267
Silhouette Coefficient: 0.113
predict info:
precision:0.093
recall:0.064
f1-score:0.070


In [0]:
from sklearn.cluster import KMeans
cv_me = KMeans(n_clusters=20, random_state=0).fit(tf)
cv_me_p=cv_me.predict(tf)
tf_me = KMeans(n_clusters=20, random_state=0).fit(tfidf)
tf_me_p=tf_me.predict(tfidf)
nmf_me = KMeans(n_clusters=20, random_state=0).fit(nmf_ma)
nmf_me_p=nmf_me.predict(nmf_ma)
lsa_me= KMeans(n_clusters=20, random_state=0).fit(X2)
lsa_me_p=lsa_me.predict(X2)


In [0]:
km_j(sampletag,cv_me_p,tf)
calculate_result(sampletag,cv_me_p)

Homogeneity: 0.033
Completeness: 0.294
V-measure: 0.060
Silhouette Coefficient: 0.580
predict info:
precision:0.070
recall:0.036
f1-score:0.006


In [0]:
km_j(sampletag,tf_me_p,tfidf)
calculate_result(sampletag,tf_me_p)

Homogeneity: 0.219
Completeness: 0.230
V-measure: 0.224
Silhouette Coefficient: -0.012
predict info:
precision:0.088
recall:0.065
f1-score:0.072


In [0]:
km_j(sampletag,nmf_me_p,nmf_ma)
calculate_result(sampletag,nmf_me_p)

Homogeneity: 0.185
Completeness: 0.236
V-measure: 0.208
Silhouette Coefficient: 0.312
predict info:
precision:0.020
recall:0.038
f1-score:0.021


In [0]:
km_j(sampletag,lsa_me_p,X2)
calculate_result(sampletag,lsa_me_p)

Homogeneity: 0.264
Completeness: 0.283
V-measure: 0.273
Silhouette Coefficient: 0.151
predict info:
precision:0.027
recall:0.045
f1-score:0.032


6.Examine the clusters. For each cluster, count the categories associated with each document. Do you notice that a category tends to mainly dominate a cluster? Which categories tend to co-occur with one another?

In [0]:
cv_ac_c = Counter(cv_ac_p)
cv_ac_c

Counter({0: 3,
         1: 41,
         2: 2,
         3: 937,
         4: 1,
         5: 1,
         6: 1,
         7: 1,
         8: 2,
         9: 1,
         10: 1,
         11: 1,
         12: 1,
         13: 1,
         14: 1,
         15: 1,
         16: 1,
         17: 1,
         18: 1,
         19: 1})

categories 3 is dominate in this cluster 

In [0]:
tf_ac_c = Counter(tf_ac_p)
tf_ac_c

Counter({0: 31,
         1: 473,
         2: 42,
         3: 120,
         4: 24,
         5: 85,
         6: 17,
         7: 30,
         8: 16,
         9: 12,
         10: 22,
         11: 13,
         12: 20,
         13: 7,
         14: 23,
         15: 15,
         16: 29,
         17: 7,
         18: 7,
         19: 7})

categories 1 is dominate in this cluster, sencond is 3, it distribute better

In [0]:
nmf_ac_c = Counter(nmf_ac_p)
nmf_ac_c

Counter({0: 27,
         1: 49,
         2: 34,
         3: 50,
         4: 310,
         5: 81,
         6: 20,
         7: 19,
         8: 90,
         9: 50,
         10: 4,
         11: 8,
         12: 11,
         13: 4,
         14: 8,
         15: 17,
         16: 26,
         17: 2,
         18: 18,
         19: 172})

categories 4 is dominate in this cluster, come after with categories 19 

In [0]:
lsa_ac_c = Counter(lsa_ac_p)
lsa_ac_c

Counter({0: 84,
         1: 46,
         2: 126,
         3: 137,
         4: 27,
         5: 47,
         6: 32,
         7: 65,
         8: 32,
         9: 26,
         10: 15,
         11: 92,
         12: 36,
         13: 58,
         14: 22,
         15: 32,
         16: 27,
         17: 48,
         18: 38,
         19: 10})

categories 3 is dominate in this cluster then is categories 2

In [0]:
cv_me_c = Counter(cv_me_p)
cv_me_c

Counter({0: 47,
         1: 1,
         2: 1,
         3: 1,
         4: 1,
         5: 1,
         6: 1,
         7: 1,
         8: 1,
         9: 1,
         10: 1,
         11: 2,
         12: 1,
         13: 1,
         14: 1,
         15: 1,
         16: 934,
         17: 1,
         18: 1,
         19: 1})

categories 16 is dominate in this cluster 

In [0]:
tf_me_c = Counter(tf_me_p)
tf_me_c

Counter({0: 28,
         1: 79,
         2: 27,
         3: 56,
         4: 36,
         5: 38,
         6: 167,
         7: 58,
         8: 50,
         9: 28,
         10: 58,
         11: 75,
         12: 34,
         13: 24,
         14: 62,
         15: 43,
         16: 17,
         17: 39,
         18: 59,
         19: 22})

categories 6 is dominate in this cluster 

In [0]:
nmf_me_c = Counter(nmf_me_p)
nmf_me_c

Counter({0: 261,
         1: 8,
         2: 21,
         3: 15,
         4: 16,
         5: 21,
         6: 6,
         7: 41,
         8: 61,
         9: 233,
         10: 30,
         11: 12,
         12: 21,
         13: 18,
         14: 4,
         15: 8,
         16: 22,
         17: 109,
         18: 40,
         19: 53})

categories 0 is dominate in this cluster come with 9 then is 17

In [0]:
lsa_me_c = Counter(lsa_me_p)
lsa_me_c

Counter({0: 45,
         1: 40,
         2: 17,
         3: 170,
         4: 10,
         5: 30,
         6: 39,
         7: 55,
         8: 31,
         9: 51,
         10: 12,
         11: 90,
         12: 48,
         13: 28,
         14: 109,
         15: 18,
         16: 44,
         17: 51,
         18: 62,
         19: 50})

categories 3 is dominate in this cluster then come with 14

<H3> Memo </H3>

7. Write up your methodology/findings in a memo. (Recall the guidelines for memos from a previous HW.) Which combination of algorithms and parameter inputs resulted in the best grouping of documents? How well does each clustering algorithm isolate the individual categories? [Think up more insights to write about!] *The memo should be a separate document from this notebook.*